In [170]:
import json
import pandas as pd
import numpy as np
import spotipy
import requests
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth
import os
from collections import defaultdict


In [171]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [172]:

username = os.environ.get("USERNAME")
client_id = os.environ.get("CLIENT_ID")
client_secret = os.environ.get("CLIENT_SECRET")
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'
data_path = f"./{username}_data"
prefix = f"{username}_"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                                client_secret,     
                                                scope=scope, 
                                                redirect_uri=redirect_uri,
                                                username=username))

In [173]:
identifiers_string = open(f"{data_path}/{prefix}Identifiers.json", "r").read()
identifiers_json = json.loads(identifiers_string)

inferences_string = open(f"{data_path}/{prefix}Inferences.json", "r").read()
inferences_json = json.loads(inferences_string)

marquee_string = open(f"{data_path}/{prefix}Marquee.json", "r").read()
marquee_json = json.loads(marquee_string)

playlist_string = open(f"{data_path}/{prefix}Playlist1.json", "r").read()
playlist_json = json.loads(playlist_string)

search_string = open(f"{data_path}/{prefix}SearchQueries.json", "r").read()
search_json = json.loads(search_string)

library_string = open(f"{data_path}/{prefix}YourLibrary.json", "r").read()
library_json = json.loads(library_string)

In [174]:
import ast
from typing import List
from os import listdir
def get_streamings(path: str) -> List[dict]:
    
    files = [f"{data_path}/{x}" for x in listdir(path)
             if x.split('.')[0][:-1] == prefix+'StreamingHistory']
    
    all_streamings = []
    
    for file in files: 
        with open(file, 'r', encoding='UTF-8') as f:
            new_streamings = ast.literal_eval(f.read())
            all_streamings += [streaming for streaming 
                               in new_streamings]
    return all_streamings

In [175]:
# from collections import defaultdict

# streamings = get_streamings(data_path)

# exceptions = 0
# tracks_to_ids = defaultdict()
# for i in range(len(streamings)):
#    track_name = streamings[i]['trackName']
#    if track_name in tracks_to_ids:
#       streamings[i]['id'] = tracks_to_ids[track_name]
#    else:
#       search = sp.search(q=f"track:{track_name}",type="track")
#       try:
#          id = search['tracks']['items'][0]['id']
#          streamings[i]['id'] = id
#       except: #          tracks_to_ids[track_name] = None

# with open(f'{data_path}/{prefix}history_with_ids', 'w') as f:
#    f.write(json.dumps(streamings, indent=2))
#
# nones = 0
# for x, y in tracks_to_ids.items():
#      if y == None:
#         nones+=1

# print(nones)

In [176]:
tracks_file = open(f'{data_path}/{prefix}history_with_ids', 'r')
tracks = json.load(tracks_file)

# drop any rows without an id
indices_to_remove = []
for i in range(len(tracks)):
   try:
      tracks[i]['id']
   except KeyError:
      indices_to_remove.append(i)
      print(tracks[i])

for x in indices_to_remove[::-1]:
   del tracks[x]
   print(x)

{'endTime': '2021-08-17 22:55', 'artistName': 'LILHUDDY', 'trackName': "Don't Freak Out (feat. iann dior, Tyson Ritter & Travis Barker)", 'msPlayed': 156000}
{'endTime': '2021-08-27 02:13', 'artistName': 'Derek Pope', 'trackName': "Don't Make Me Feel Alone", 'msPlayed': 211099}
{'endTime': '2021-08-30 07:19', 'artistName': 'Foals', 'trackName': "I'm Done With The World (& It's Done With Me)", 'msPlayed': 182030}
{'endTime': '2021-09-02 16:36', 'artistName': 'Murge', 'trackName': "Can't Hurt Me Now", 'msPlayed': 359572}
{'endTime': '2021-09-03 01:27', 'artistName': 'James Blake', 'trackName': "Can't Believe The Way We Flow", 'msPlayed': 267200}
{'endTime': '2021-09-03 02:19', 'artistName': 'John Rzeznik', 'trackName': 'I\'m Still Here (Jim\'s Theme) - From "Treasure Planet"/Soundtrack Version', 'msPlayed': 251466}
{'endTime': '2021-09-10 01:04', 'artistName': 'The Tiny Meat Gang Podcast', 'trackName': 'Ep 202 - Cody got a haircut ft. Ben Cahn and Emil DeRosa', 'msPlayed': 847760}
{'endT

In [177]:
def get_features(track_id: str) -> dict:
    try:
        features = sp.audio_features([track_id])
        return features[0]
    except:
        return None

In [178]:
# id_to_features = defaultdict()
# for i in range(len(tracks)):
#    print(tracks[i])
#    id = tracks[i]['id']
#    if id in id_to_features:
#       tracks[i]['features'] = id_to_features[id]
#    else:
#       features = get_features(id)
#       tracks[i]['features'] = features
#       id_to_features[id] = features
# with open(f'{data_path}/{prefix}history_with_features', 'w') as f:
#    f.write(json.dumps(tracks, indent=2))

In [179]:
# AFTER RUNNING THE TWO QUERIES THIS SHOULD BE THE FINAL FILE WITH SONG IDS AND FEATURES OF EACH
tracks_file = open(f'{data_path}/{prefix}history_with_features', 'r')
tracks = json.load(tracks_file)

# flatten dictionary
dataset_df = pd.DataFrame(tracks)
dataset_df = pd.json_normalize(tracks, sep='_')
dataset_df = dataset_df.drop("features", 1)
dataset_df = dataset_df.rename(columns={"endTime": "end_time", "artistName": "artist_name", "trackName": "track_name", "msPlayed": "ms_played"})

# drop null ids or features
dataset_df = dataset_df.dropna(subset=dataset_df.columns.array[4:])
dataset_df

/var/folders/xy/7p54kpbs7f16djjx60_rtvch0000gn/T/ipykernel_47062/1115805306.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dataset_df = dataset_df.drop("features", 1)


,end_time,artist_name,track_name,ms_played,id,features_danceability,features_energy,features_key,features_loudness,features_mode,...,features_liveness,features_valence,features_tempo,features_type,features_id,features_uri,features_track_href,features_analysis_url,features_duration_ms,features_time_signature
0,2021-08-17 22:43,Trippie Redd,LEADERS,160993,4D7NrSeqkTarBrJ80b2sBc,0.950,0.654,0.0,-5.895,0.0,...,0.1380,0.886,132.936,audio_features,4D7NrSeqkTarBrJ80b2sBc,spotify:track:4D7NrSeqkTarBrJ80b2sBc,https://api.spotify.com/v1/tracks/4D7NrSeqkTar...,https://api.spotify.com/v1/audio-analysis/4D7N...,194844.0,4.0
1,2021-08-17 22:47,Trippie Redd,FROZEN OCEAN,196169,5ypQ8scn8g2l0341ErES0B,0.314,0.762,2.0,-3.136,1.0,...,0.1160,0.310,134.582,audio_features,5ypQ8scn8g2l0341ErES0B,spotify:track:5ypQ8scn8g2l0341ErES0B,https://api.spotify.com/v1/tracks/5ypQ8scn8g2l...,https://api.spotify.com/v1/audio-analysis/5ypQ...,204233.0,4.0
2,2021-08-17 22:49,WILLOW,t r a n s p a r e n t s o u l feat. Travis Barker,168398,1QL7nSDZCwZMnbisV4KOXt,0.390,0.830,11.0,-5.201,0.0,...,0.1560,0.191,90.021,audio_features,1QL7nSDZCwZMnbisV4KOXt,spotify:track:1QL7nSDZCwZMnbisV4KOXt,https://api.spotify.com/v1/tracks/1QL7nSDZCwZM...,https://api.spotify.com/v1/audio-analysis/1QL7...,168030.0,4.0
3,2021-08-17 22:52,Machine Gun Kelly,I Think I'm OKAY (with YUNGBLUD & Travis Barker),169397,2gTdDMpNxIRFSiu7HutMCg,0.628,0.744,7.0,-4.718,1.0,...,0.3130,0.277,119.921,audio_features,2gTdDMpNxIRFSiu7HutMCg,spotify:track:2gTdDMpNxIRFSiu7HutMCg,https://api.spotify.com/v1/tracks/2gTdDMpNxIRF...,https://api.spotify.com/v1/audio-analysis/2gTd...,169397.0,4.0
4,2021-08-17 22:58,Trippie Redd,PILL BREAKER (feat. blackbear & Machine Gun Ke...,177058,7jjnn6K7p5MgOFuqLypAcM,0.527,0.790,2.0,-6.268,1.0,...,0.3700,0.236,153.034,audio_features,7jjnn6K7p5MgOFuqLypAcM,spotify:track:7jjnn6K7p5MgOFuqLypAcM,https://api.spotify.com/v1/tracks/7jjnn6K7p5Mg...,https://api.spotify.com/v1/audio-analysis/7jjn...,177059.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12753,2021-08-17 22:29,Trippie Redd,RED SKY (feat. Machine Gun Kelly),152149,0yFfjXtSDkr81WVAPPS1Q4,0.472,0.769,7.0,-4.198,1.0,...,0.2390,0.587,159.999,audio_features,0yFfjXtSDkr81WVAPPS1Q4,spotify:track:0yFfjXtSDkr81WVAPPS1Q4,https://api.spotify.com/v1/tracks/0yFfjXtSDkr8...,https://api.spotify.com/v1/audio-analysis/0yFf...,152149.0,4.0
12754,2021-08-17 22:33,Trippie Redd,MEGLADON,194992,5Kq6LIFZsVD99rb7iT0QTG,0.453,0.739,7.0,-4.977,1.0,...,0.6940,0.258,139.962,audio_features,5Kq6LIFZsVD99rb7iT0QTG,spotify:track:5Kq6LIFZsVD99rb7iT0QTG,https://api.spotify.com/v1/tracks/5Kq6LIFZsVD9...,https://api.spotify.com/v1/audio-analysis/5Kq6...,194992.0,4.0
12755,2021-08-17 22:35,Trippie Redd,SAVE YOURSELF,160729,6FhB5ZyT4JsIJ0ozDlXCIp,0.503,0.921,8.0,-5.907,0.0,...,0.2600,0.359,121.983,audio_features,6FhB5ZyT4JsIJ0ozDlXCIp,spotify:track:6FhB5ZyT4JsIJ0ozDlXCIp,https://api.spotify.com/v1/tracks/6FhB5ZyT4JsI...,https://api.spotify.com/v1/audio-analysis/6FhB...,221160.0,4.0
12756,2021-08-17 22:38,Trippie Redd,DREAMER,170000,4yZEX3NsnXfobmZ0m8ZO4Y,0.723,0.751,9.0,-5.233,0.0,...,0.0680,0.642,126.999,audio_features,4yZEX3NsnXfobmZ0m8ZO4Y,spotify:track:4yZEX3NsnXfobmZ0m8ZO4Y,https://api.spotify.com/v1/tracks/4yZEX3NsnXfo...,https://api.spotify.com/v1/audio-analysis/4yZE...,190849.0,4.0
